In [1]:
import pandas as pd
import numpy as np

In [2]:
num_of_criteria = 6
criteria = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
number_of_evaluators = 2

In [3]:
path = '../g-aph-matrix-results.xlsx'
results_df = pd.read_excel(path, header=0)
results_df = results_df.fillna(0)
results_df

,C-R,EZ-R,VJZ-R,JZ-R,SZ-R,PZ,SZ-L,JZ-L,VJZ-L,EZ-L,C-L
0,C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,C2
1,C1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,C3
2,C1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,C4
3,C1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,C5
4,C1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,C6
5,C2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,C3
6,C2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C4
7,C2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C5
8,C2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,C6
9,C3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C4


In [4]:
ling_scale = ['Podjednaka znacajnost', 'Srednja znacajnost', 'Jaka znacajnost', 'Veoma jaka znacajnost', 'Ekstremna znacajnost']
scale = {'Linguistic Scale': ling_scale, 'Code': ['PZ', 'SZ', 'JZ', 'VJZ', 'EZ'], 'Grey Numbers': [[1, 2], [2, 4], [4, 6], [6, 8], [8, 10]]}
scale_df = pd.DataFrame(data=scale)
scale_df

,Linguistic Scale,Code,Grey Numbers
0,Podjednaka znacajnost,PZ,"[1, 2]"
1,Srednja znacajnost,SZ,"[2, 4]"
2,Jaka znacajnost,JZ,"[4, 6]"
3,Veoma jaka znacajnost,VJZ,"[6, 8]"
4,Ekstremna znacajnost,EZ,"[8, 10]"


In [5]:
def fill_diagonal(matrix, fill_value):
    matrix_dim = matrix.shape[0]
    for i in range(matrix_dim):
        for j in range(matrix_dim):
            if i == j:
                matrix[i,j] = fill_value

def print_grey_num(grey_num):
    return '[ ' + str(grey_num[0]) + ', ' + str(grey_num[1]) + ' ]'
                
def print_matrix(matrix):
    matrix_dim = matrix.shape[0]
    for i in range(matrix_dim):
        for j in range(matrix_dim):
            print(print_grey_num(matrix[i,j,:]), end =" ")
        print("\n")

In [6]:
aph_matrix = np.zeros((num_of_criteria, num_of_criteria, 2), dtype=float)
fill_diagonal(aph_matrix, [1.0, 1.0])
print_matrix(aph_matrix)

[ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] 



In [7]:
def get_aph_matrix_indices_for_criteria(right_criteria_str, left_criteria_str):
    # remove 'C' from the begining
    right_criteria = right_criteria_str[1:len(right_criteria_str):1]
    left_criteria = left_criteria_str[1:len(left_criteria_str):1]
    return int(right_criteria)-1, int(left_criteria)-1

def get_reciprocal_grey_num(grey_num):
    return np.array([1/grey_num[1], 1/grey_num[0]])

def update_aph_matrix(grey_num_for_right_c, row_ind, col_ind):
    grey_num_for_left_c = get_reciprocal_grey_num(grey_num_for_right_c)
    aph_matrix[row_ind, col_ind] = np.around(grey_num_for_right_c, 3)
    aph_matrix[col_ind, row_ind] = np.around(grey_num_for_left_c, 3)

In [8]:
def get_code_and_side(axes_label):
    split_label = axes_label.split('-')
    if len(split_label) > 1:
        return split_label[0], split_label[1] #JZ, R
    return split_label[0], None # PZ

def get_grey_number_for_code(ling_scale_code):
    grey_num = list(scale_df[scale_df['Code'] == ling_scale_code]['Grey Numbers'])[0]
    return np.array(grey_num)

def left_c_is_more_important(side):
    return side != None and side == 'L'

def calc_grey_num_for_right_c_based_on_comparison(criteria_comparison):
    criteria_comparison = criteria_comparison[criteria_comparison!=0]
    length_of_series = criteria_comparison.shape[0]
    axes = criteria_comparison.axes[0]
    
    grey_num_for_right_criteria = np.array([0.0, 0.0])
    
    for i in range(1, length_of_series-1): #skip first and last 
        axes_label = axes[i] #JZ-R
        ling_scale_code, more_important_c_side = get_code_and_side(axes_label)
        grey_number_for_code = get_grey_number_for_code(ling_scale_code)
        
        if left_c_is_more_important(more_important_c_side):
            grey_number_for_code = get_reciprocal_grey_num(grey_number_for_code)
       
        num_of_evaluation_with_code = int(criteria_comparison[i])
        grey_num_for_right_criteria +=  grey_number_for_code * num_of_evaluation_with_code
    
    return grey_num_for_right_criteria / number_of_evaluators

In [9]:
num_of_comparisons = results_df.shape[0]

for i in range(num_of_comparisons):
    criteria_comparison = results_df.loc[i,:]
    right_criteria = criteria_comparison['C-R']
    left_criteria = criteria_comparison['C-L']
    grey_num_for_right_c = calc_grey_num_for_right_c_based_on_comparison(criteria_comparison)
    row_ind, col_ind = get_aph_matrix_indices_for_criteria(right_criteria, left_criteria)
    update_aph_matrix(grey_num_for_right_c, row_ind, col_ind)


In [10]:
print_matrix(aph_matrix)

[ 1.0, 1.0 ] [ 0.1, 0.125 ] [ 1.05, 2.062 ] [ 0.625, 1.25 ] [ 5.0, 7.0 ] [ 0.133, 0.188 ] 

[ 8.0, 10.0 ] [ 1.0, 1.0 ] [ 5.0, 7.0 ] [ 8.0, 10.0 ] [ 8.0, 10.0 ] [ 4.0, 6.0 ] 

[ 0.485, 0.952 ] [ 0.143, 0.2 ] [ 1.0, 1.0 ] [ 3.0, 4.0 ] [ 4.5, 6.0 ] [ 2.125, 3.25 ] 

[ 0.8, 1.6 ] [ 0.1, 0.125 ] [ 0.25, 0.333 ] [ 1.0, 1.0 ] [ 2.083, 3.125 ] [ 0.146, 0.208 ] 

[ 0.143, 0.2 ] [ 0.1, 0.125 ] [ 0.167, 0.222 ] [ 0.32, 0.48 ] [ 1.0, 1.0 ] [ 0.112, 0.146 ] 

[ 5.333, 7.5 ] [ 0.167, 0.25 ] [ 0.308, 0.471 ] [ 4.8, 6.857 ] [ 6.857, 8.889 ] [ 1.0, 1.0 ] 



In [11]:
def get_normalize_grey_num(row_ind, col_ind):
    grey_num_comparison = aph_matrix[row_ind,col_ind]
    grey_num_lower_bound = grey_num_comparison[0]
    grey_num_upper_bound = grey_num_comparison[1]

    normalizing_factor = normalizing_factors[col_ind]
    
    low_bound_norm = 2 * grey_num_lower_bound / normalizing_factor
    upper_bound_norm = 2 * grey_num_upper_bound / normalizing_factor
    normalized_grey_num = [low_bound_norm, upper_bound_norm]
    
    return np.around(normalized_grey_num, 3)

In [12]:
# normalize the grey comparison matrix
normalized_aph_matrix = np.zeros((num_of_criteria, num_of_criteria, 2), dtype=float)
fill_diagonal(normalized_aph_matrix, [1.0, 1.0])

In [13]:
# calculate normalizing factors 
normalizing_factors = []

for col in range(num_of_criteria):
    normalizing_col = np.ravel(aph_matrix[:,col,:])
    normalizing_factors.append(sum(normalizing_col))
    
normalizing_factors

[37.013, 3.435, 18.863000000000003, 41.332, 63.45399999999999, 18.308]

In [14]:
for i in range(num_of_criteria):
    for j in range(num_of_criteria):
        norm_grey_num = get_normalize_grey_num(i, j)
        normalized_aph_matrix[i,j] = norm_grey_num

print_matrix(normalized_aph_matrix)

[ 0.054, 0.054 ] [ 0.058, 0.073 ] [ 0.111, 0.219 ] [ 0.03, 0.06 ] [ 0.158, 0.221 ] [ 0.015, 0.021 ] 

[ 0.432, 0.54 ] [ 0.582, 0.582 ] [ 0.53, 0.742 ] [ 0.387, 0.484 ] [ 0.252, 0.315 ] [ 0.437, 0.655 ] 

[ 0.026, 0.051 ] [ 0.083, 0.116 ] [ 0.106, 0.106 ] [ 0.145, 0.194 ] [ 0.142, 0.189 ] [ 0.232, 0.355 ] 

[ 0.043, 0.086 ] [ 0.058, 0.073 ] [ 0.027, 0.035 ] [ 0.048, 0.048 ] [ 0.066, 0.098 ] [ 0.016, 0.023 ] 

[ 0.008, 0.011 ] [ 0.058, 0.073 ] [ 0.018, 0.024 ] [ 0.015, 0.023 ] [ 0.032, 0.032 ] [ 0.012, 0.016 ] 

[ 0.288, 0.405 ] [ 0.097, 0.146 ] [ 0.033, 0.05 ] [ 0.232, 0.332 ] [ 0.216, 0.28 ] [ 0.109, 0.109 ] 



In [15]:
# calculate grey weights for criteria
grey_weights_for_criteria = []

for c in range(num_of_criteria):
    weight_row_low = np.ravel(normalized_aph_matrix[c,:,0])
    weight_factor_low = sum(weight_row_low)
    weight_row_upp = np.ravel(normalized_aph_matrix[c,:,1])
    weight_factor_upp = sum(weight_row_upp)
    
    weight_lower_bound = weight_factor_low / num_of_criteria
    weight_upper_bound = weight_factor_upp / num_of_criteria
    grey_weight = np.array([weight_lower_bound, weight_upper_bound])
    
    grey_weights_for_criteria.append(np.around(grey_weight, 3))
    
grey_weights_for_criteria

[array([0.071, 0.108]),
 array([0.437, 0.553]),
 array([0.122, 0.168]),
 array([0.043, 0.06 ]),
 array([0.024, 0.03 ]),
 array([0.162, 0.22 ])]

In [16]:
# whitenization of the grey weight with alpha=0.5

crisp_weight_numbers = [(w[0] + w[1])/2 for w in grey_weights_for_criteria]
crisp_weight_numbers = [round(w, 3) for w in crisp_weight_numbers]
crisp_weight_numbers

[0.09, 0.495, 0.145, 0.052, 0.027, 0.191]

In [17]:
criteria_importance_weights = {'Criteria': criteria, 'Importance weight': crisp_weight_numbers}
criteria_importance_weights_df = pd.DataFrame(data=criteria_importance_weights)
criteria_importance_weights_df

,Criteria,Importance weight
0,C1,0.090
1,C2,0.495
2,C3,0.145
3,C4,0.052
4,C5,0.027
5,C6,0.191
